<a href="https://colab.research.google.com/github/daliarod96/scraping-Kurt-Vile-lyrics-using-Genius-API/blob/main/processing-kv-lyrics-dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I will be processing the Kurt Vile lyrics dataset so I can do some visualizations and sentiment analysis on it.


In [207]:
import pandas as pd
import numpy as np
import re

In [208]:
# Import my lyrics dataframe
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/kv-lyrics-unprocessed.csv")

In [209]:
df.head(11)

,Unnamed: 0,song-name,lyrics,release-year,genius-path
0,0,Air Bud,"Strange, strange days inside My mind is daily changing And it's charged by The shifting tones of time flying Just in case you were wondering where I Was Was Strange Strange, strange days inside My mind is daily changing As result of the black cloud coming down Just in case you were watching the Horizon Horizon Horizon (We want out and we want it now) (We wanna take it outta town) (We want out and we want it now) (We wanna take it outta town) Strange, strange days inside",2013,/Kurt-vile-air-bud-lyrics
1,1,All in a Daze Work,"[Verse 1] Chinese charms and well-tanked in Burbank Strumming unsuccessfully but moreso just pressin' keys and Hey Jack of diamonds all jack of horse Hey man I believe I'm floating and off course of course Guess I got my mind well twisted didn’t I, well… [Chorus] All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a… all in a… all in a daze work [Verse 2] She come flying through like a traveling gypsy show tornado Leaving us lying there bleedin’ Cut wide open, hearts broken Guess I got my mind well twisted didn’t I, well… Chorus All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a… all in a… all in a daze work",2015,/Kurt-vile-all-in-a-daze-work-lyrics
2,2,Amplifier,"Don't just stand [?] your head Don't sit [?] [?] Turn around, to the trees Toss your key 'Cause this is home This is home [?] alone The [?] That you got to know I don't mean to cause you suffering [?] 'Cause I got nothing in my head [?] To tell a [?] He was alone He was alone Don't sit still[?} [?] head Don't sit [?] [?] Upside down Turn around, towards the amplifier [?] Understand my command Free[?] your keys 'Cause this is home This is home This is home Home Home",2009,/Kurt-vile-amplifier-lyrics
3,3,Baby’s Arms,"There has been but one true love In my baby’s arms, in my baby’s arms And I got the hands to hold onto them I get sick of just about everyone And I hide in my baby’s arms, my baby’s arms 'Cause except for her, you know, as I've implied I will never ever ever be alone 'Cause it’s all in my baby’s hands Shiny, shiny secret stones In my baby’s hands In my baby’s hands I get sick of just about everyone And I hide in my baby’s arms Shrink myself just like a Tom Thumb And I hide in my baby’s hands Hide in my baby’s hands 'Cause except for her There just ain't nothing to latch onto Yeah, yeah, yeah There has been but one true love In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms",2011,/Kurt-vile-babys-arms-lyrics
4,4,Bad Omens,False,2015,/Kurt-vile-bad-omens-lyrics
5,5,Bassackwards,"[Intro: Spoken] What's in real...what's in real time? Ha ha ha [?] I was on the beach, but I was thinking about the bay Got to the bay, but by then I was far away I was on the ground but looking straight into the sun But the sun went down and I couldn't find another one For awhile For because it was an all-burning feeling in my chest To fill the void Of a long night unwatched by, well, the sun Until the morn Until when, well, the sun's reborn And so am I From all the scorn buried deep Within the psyche of my soul I was standing down but I was also on the run In my mind I was on the radio talking with a friend of mine There wasn't no format because well, we like it like that He was out of his mind and I was way out of mine Then everything went backwards With words coming out Bassackwards And I appreciate him to the utmost degree",2018,/Kurt-vile-bassackwards-lyrics
6,6,Beach On The Moon (Recycled Lyrics),"I got a freeway in mind, let go of my head Walk down my line, better be sure you'll be dead You get up on top, these days you gotta be a low-life drifter so Slither up just like a snake upon a spiral staircase Got a freeway in mind, let go of my head Walk down my line, be

In [210]:
# I'm dropping this useless column
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [211]:
# I'm going to drop row 10 because it's not a song.
df.drop([10], inplace=True)
df.reset_index(drop=True, inplace=True)

In [212]:
df.head(11)

,song-name,lyrics,release-year,genius-path
0,Air Bud,"Strange, strange days inside My mind is daily changing And it's charged by The shifting tones of time flying Just in case you were wondering where I Was Was Strange Strange, strange days inside My mind is daily changing As result of the black cloud coming down Just in case you were watching the Horizon Horizon Horizon (We want out and we want it now) (We wanna take it outta town) (We want out and we want it now) (We wanna take it outta town) Strange, strange days inside",2013,/Kurt-vile-air-bud-lyrics
1,All in a Daze Work,"[Verse 1] Chinese charms and well-tanked in Burbank Strumming unsuccessfully but moreso just pressin' keys and Hey Jack of diamonds all jack of horse Hey man I believe I'm floating and off course of course Guess I got my mind well twisted didn’t I, well… [Chorus] All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a… all in a… all in a daze work [Verse 2] She come flying through like a traveling gypsy show tornado Leaving us lying there bleedin’ Cut wide open, hearts broken Guess I got my mind well twisted didn’t I, well… Chorus All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a… all in a… all in a daze work",2015,/Kurt-vile-all-in-a-daze-work-lyrics
2,Amplifier,"Don't just stand [?] your head Don't sit [?] [?] Turn around, to the trees Toss your key 'Cause this is home This is home [?] alone The [?] That you got to know I don't mean to cause you suffering [?] 'Cause I got nothing in my head [?] To tell a [?] He was alone He was alone Don't sit still[?} [?] head Don't sit [?] [?] Upside down Turn around, towards the amplifier [?] Understand my command Free[?] your keys 'Cause this is home This is home This is home Home Home",2009,/Kurt-vile-amplifier-lyrics
3,Baby’s Arms,"There has been but one true love In my baby’s arms, in my baby’s arms And I got the hands to hold onto them I get sick of just about everyone And I hide in my baby’s arms, my baby’s arms 'Cause except for her, you know, as I've implied I will never ever ever be alone 'Cause it’s all in my baby’s hands Shiny, shiny secret stones In my baby’s hands In my baby’s hands I get sick of just about everyone And I hide in my baby’s arms Shrink myself just like a Tom Thumb And I hide in my baby’s hands Hide in my baby’s hands 'Cause except for her There just ain't nothing to latch onto Yeah, yeah, yeah There has been but one true love In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms",2011,/Kurt-vile-babys-arms-lyrics
4,Bad Omens,False,2015,/Kurt-vile-bad-omens-lyrics
5,Bassackwards,"[Intro: Spoken] What's in real...what's in real time? Ha ha ha [?] I was on the beach, but I was thinking about the bay Got to the bay, but by then I was far away I was on the ground but looking straight into the sun But the sun went down and I couldn't find another one For awhile For because it was an all-burning feeling in my chest To fill the void Of a long night unwatched by, well, the sun Until the morn Until when, well, the sun's reborn And so am I From all the scorn buried deep Within the psyche of my soul I was standing down but I was also on the run In my mind I was on the radio talking with a friend of mine There wasn't no format because well, we like it like that He was out of his mind and I was way out of mine Then everything went backwards With words coming out Bassackwards And I appreciate him to the utmost degree",2018,/Kurt-vile-bassackwards-lyrics
6,Beach On The Moon (Recycled Lyrics),"I got a freeway in mind, let go of my head Walk down my line, better be sure you'll be dead You get up on top, these days you gotta be a low-life drifter so Slither up just like a snake upon a spiral staircase Got a freeway in mind, let go of my head Walk down my line, better be sure you'll be de

In [213]:
# Take a closer look at the lyrics
pd.set_option('display.max_colwidth', None)
df

,song-name,lyrics,release-year,genius-path
0,Air Bud,"Strange, strange days inside My mind is daily changing And it's charged by The shifting tones of time flying Just in case you were wondering where I Was Was Strange Strange, strange days inside My mind is daily changing As result of the black cloud coming down Just in case you were watching the Horizon Horizon Horizon (We want out and we want it now) (We wanna take it outta town) (We want out and we want it now) (We wanna take it outta town) Strange, strange days inside",2013,/Kurt-vile-air-bud-lyrics
1,All in a Daze Work,"[Verse 1] Chinese charms and well-tanked in Burbank Strumming unsuccessfully but moreso just pressin' keys and Hey Jack of diamonds all jack of horse Hey man I believe I'm floating and off course of course Guess I got my mind well twisted didn’t I, well… [Chorus] All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a… all in a… all in a daze work [Verse 2] She come flying through like a traveling gypsy show tornado Leaving us lying there bleedin’ Cut wide open, hearts broken Guess I got my mind well twisted didn’t I, well… Chorus All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a daze work, and All in a… all in a… all in a daze work",2015,/Kurt-vile-all-in-a-daze-work-lyrics
2,Amplifier,"Don't just stand [?] your head Don't sit [?] [?] Turn around, to the trees Toss your key 'Cause this is home This is home [?] alone The [?] That you got to know I don't mean to cause you suffering [?] 'Cause I got nothing in my head [?] To tell a [?] He was alone He was alone Don't sit still[?} [?] head Don't sit [?] [?] Upside down Turn around, towards the amplifier [?] Understand my command Free[?] your keys 'Cause this is home This is home This is home Home Home",2009,/Kurt-vile-amplifier-lyrics
3,Baby’s Arms,"There has been but one true love In my baby’s arms, in my baby’s arms And I got the hands to hold onto them I get sick of just about everyone And I hide in my baby’s arms, my baby’s arms 'Cause except for her, you know, as I've implied I will never ever ever be alone 'Cause it’s all in my baby’s hands Shiny, shiny secret stones In my baby’s hands In my baby’s hands I get sick of just about everyone And I hide in my baby’s arms Shrink myself just like a Tom Thumb And I hide in my baby’s hands Hide in my baby’s hands 'Cause except for her There just ain't nothing to latch onto Yeah, yeah, yeah There has been but one true love In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms In my baby’s arms",2011,/Kurt-vile-babys-arms-lyrics
4,Bad Omens,False,2015,/Kurt-vile-bad-omens-lyrics
...,...,...,...,...
123,Wheelhouse,"[Verse 1] Sometimes I talk too much but I gotta get it out But I don’t wanna talk, I only wanna listen My baby talks soft, my ears are always ringing now Humming a sad song when I’m alone But you gotta be alone to figure things out sometimes Be alone, when even in a crowd of friends and not so Sometimes of whom you just can’t distinguish but Thank god for the former, yeah [Chorus] Some bow down a hundred times a day or more To find a way, to get back down to the temple one day Find the temple, to bask in the glory of Roll around on a floor of furry carpet then Sleep soundly for the first time in forever and Breathing deep inside [Verse 2] There’s a desert down below the earth’s core A hidden staircase in the house that you reside in A little something from the cupboard over there A little medicine, it’s a medication situation [Chorus] Some bow down a hundred times a day or more To find a way, to get back down to the temple one day Find the temple, to bask in the glory of Roll around on a floor of furry carpet then Sleep soundly for the first time in forever and Breathing deep inside",2015,/Kurt-vile-wheelhouse-lyrics
124,White Riffs,False,2009,/Kurt-vile-white-riffs-lyrics
1

Lyrics procesing:
*   remove stuff in brackets 
*   remove punctuation
*   remove parentheses
*   turn all letters to lowercase


In [214]:
sample = "[?][2:43] [Chorus][x3](One, two, three, four) [Verse 1] When nobody calls you on the phone Don't break your bones over it You'll only leave yourself lonesome To show for it When nobody calls you on the phone Don't break your bones Dyin' dead in a black sea of tears over me [Verse 2] When you're runnin' on the run But you run, run too far (Tomfoolin' and buckwild) Deep in the country wildcats get shot at When you're runnin' on the run But you run too far Where the rednecks raise hell With the shotgun shell (blast!) [Instrumental] [Verse 3] When nobody calls me on the phone Won't break my bones over it I don't believe myself lonesome To show for that When nobody calls me on the phone Won't break my bones Dyin' dead in a black sea of tears over me (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now)"
print(sample)

[?][2:43] [Chorus][x3](One, two, three, four) [Verse 1] When nobody calls you on the phone Don't break your bones over it You'll only leave yourself lonesome To show for it When nobody calls you on the phone Don't break your bones Dyin' dead in a black sea of tears over me [Verse 2] When you're runnin' on the run But you run, run too far (Tomfoolin' and buckwild) Deep in the country wildcats get shot at When you're runnin' on the run But you run too far Where the rednecks raise hell With the shotgun shell (blast!) [Instrumental] [Verse 3] When nobody calls me on the phone Won't break my bones over it I don't believe myself lonesome To show for that When nobody calls me on the phone Won't break my bones Dyin' dead in a black sea of tears over me (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now)


In [215]:
# remove stuff in brackets
sample = re.sub("\[[^\]]*\]",'', sample )
sample

" (One, two, three, four)  When nobody calls you on the phone Don't break your bones over it You'll only leave yourself lonesome To show for it When nobody calls you on the phone Don't break your bones Dyin' dead in a black sea of tears over me  When you're runnin' on the run But you run, run too far (Tomfoolin' and buckwild) Deep in the country wildcats get shot at When you're runnin' on the run But you run too far Where the rednecks raise hell With the shotgun shell (blast!)   When nobody calls me on the phone Won't break my bones over it I don't believe myself lonesome To show for that When nobody calls me on the phone Won't break my bones Dyin' dead in a black sea of tears over me (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now) (Over me, over me, over me now)"

In [216]:
# remove anything that's not a word
sample = re.sub("[^\w'\s]",'', sample)
sample

" One two three four  When nobody calls you on the phone Don't break your bones over it You'll only leave yourself lonesome To show for it When nobody calls you on the phone Don't break your bones Dyin' dead in a black sea of tears over me  When you're runnin' on the run But you run run too far Tomfoolin' and buckwild Deep in the country wildcats get shot at When you're runnin' on the run But you run too far Where the rednecks raise hell With the shotgun shell blast   When nobody calls me on the phone Won't break my bones over it I don't believe myself lonesome To show for that When nobody calls me on the phone Won't break my bones Dyin' dead in a black sea of tears over me Over me over me over me now Over me over me over me now Over me over me over me now Over me over me over me now Over me over me over me now Over me over me over me now"

In [217]:
# turn all letters to lowercase
sample = sample.lower()
sample

" one two three four  when nobody calls you on the phone don't break your bones over it you'll only leave yourself lonesome to show for it when nobody calls you on the phone don't break your bones dyin' dead in a black sea of tears over me  when you're runnin' on the run but you run run too far tomfoolin' and buckwild deep in the country wildcats get shot at when you're runnin' on the run but you run too far where the rednecks raise hell with the shotgun shell blast   when nobody calls me on the phone won't break my bones over it i don't believe myself lonesome to show for that when nobody calls me on the phone won't break my bones dyin' dead in a black sea of tears over me over me over me over me now over me over me over me now over me over me over me now over me over me over me now over me over me over me now over me over me over me now"

Now do all of the above to the entire dataframe.



In [218]:
df['lyrics'] = df['lyrics'].apply(lambda x: re.sub("\[[^\]]*\]",'', x))

In [219]:
df['lyrics'] = df['lyrics'].apply(lambda x: re.sub("[^\w'\s]",'', x))

In [220]:
df['lyrics'] = df['lyrics'].apply(lambda x: x.lower())

In [222]:
# change instrumental songs' lyrics from 'false' to "instrumental"
mask = df['lyrics'] == 'false'
df['lyrics'][mask] = 'instrumental'

<ipython-input-222-cde5323db8f6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lyrics'][mask] = 'instrumental'


In [223]:
df

,song-name,lyrics,release-year,genius-path
0,Air Bud,strange strange days inside my mind is daily changing and it's charged by the shifting tones of time flying just in case you were wondering where i was was strange strange strange days inside my mind is daily changing as result of the black cloud coming down just in case you were watching the horizon horizon horizon we want out and we want it now we wanna take it outta town we want out and we want it now we wanna take it outta town strange strange days inside,2013,/Kurt-vile-air-bud-lyrics
1,All in a Daze Work,chinese charms and welltanked in burbank strumming unsuccessfully but moreso just pressin' keys and hey jack of diamonds all jack of horse hey man i believe i'm floating and off course of course guess i got my mind well twisted didnt i well all in a daze work and all in a daze work and all in a daze work and all in a daze work and all in a daze work and all in a all in a all in a daze work she come flying through like a traveling gypsy show tornado leaving us lying there bleedin cut wide open hearts broken guess i got my mind well twisted didnt i well chorus all in a daze work and all in a daze work and all in a daze work and all in a daze work and all in a daze work and all in a all in a all in a daze work,2015,/Kurt-vile-all-in-a-daze-work-lyrics
2,Amplifier,don't just stand your head don't sit turn around to the trees toss your key 'cause this is home this is home alone the that you got to know i don't mean to cause you suffering 'cause i got nothing in my head to tell a he was alone he was alone don't sit still head don't sit upside down turn around towards the amplifier understand my command free your keys 'cause this is home this is home this is home home home,2009,/Kurt-vile-amplifier-lyrics
3,Baby’s Arms,there has been but one true love in my babys arms in my babys arms and i got the hands to hold onto them i get sick of just about everyone and i hide in my babys arms my babys arms 'cause except for her you know as i've implied i will never ever ever be alone 'cause its all in my babys hands shiny shiny secret stones in my babys hands in my babys hands i get sick of just about everyone and i hide in my babys arms shrink myself just like a tom thumb and i hide in my babys hands hide in my babys hands 'cause except for her there just ain't nothing to latch onto yeah yeah yeah there has been but one true love in my babys arms in my babys arms in my babys arms in my babys arms in my babys arms in my babys arms in my babys arms,2011,/Kurt-vile-babys-arms-lyrics
4,Bad Omens,instrumental,2015,/Kurt-vile-bad-omens-lyrics
...,...,...,...,...
123,Wheelhouse,sometimes i talk too much but i gotta get it out but i dont wanna talk i only wanna listen my baby talks soft my ears are always ringing now humming a sad song when im alone but you gotta be alone to figure things out sometimes be alone when even in a crowd of friends and not so sometimes of whom you just cant distinguish but thank god for the former yeah some bow down a hundred times a day or more to find a way to get back down to the temple one day find the temple to bask in the glory of roll around on a floor of furry carpet then sleep soundly for the first time in forever and breathing deep inside theres a desert down below the earths core a hidden staircase in the house that you reside in a little something from the cupboard over there a little medicine its a medication situation some bow down a hundred times a day or more to find a way to get back down to the temple one day find the temple to bask in the glory of roll around on a floor of furry carpet then sleep soundly for the first time in forever and breathing deep inside,2015,/Kurt-vile-wheelhouse-lyrics
124,White Riffs,instrumental,2009,/Kurt-vile-white-riffs-lyrics
125,Wild Imagination,im looking at you but its only a picture so i take that back but it aint really a picture its just an image on a screen you can imagine if i was though right just like i can

In [224]:
df['release-year'].unique()

array([2013, 2015, 2009, 2011, 2018,    0, 2016, 2022, 2020, 2010, 2019,
       2021, 2008])

I am only interested in Kurt Vile's officially released music so I am going to delete songs with release date 0.

In [225]:
mask = df['release-year'] == 0
df[mask]

,song-name,lyrics,release-year,genius-path
7,Best Love,the best kind of love is found when you're not looking it comes and goes who plant the rose that you least expect it to it's only true and true if you don't have to ask for it true friends come disguised and go beyond the call of duty take for example recognize my cousin you'll see he's like a brother to me i got a good friend that looks up for me like a father figure to distant relatives sometimes i lay awake at night turning 'cause i lost my best friend if you had a true friend i'm sorry and you know i feel for you me i lost someone like a doctor so its deeper than i talk to you it makes me feel like a part of me is dead too take into account she's my mother she looks to me to protect younger kin it's a family thing every son would fight for your sisters and brothers you're the man of the house but daddy's gonna stop at nothing,0,/Kurt-vile-best-love-lyrics
9,Black Hands,instrumental,0,/Kurt-vile-black-hands-lyrics
13,Breathin’ Out,last night i was high i heard you say it was my body or my brain that said my soul was dead you can only argue with yourself for so long through my body or my brain i got myself insane hey yeah you got yourself all mixed up didn't you yeah didn't you yeah hey yeah you forgot that you gotta be taking in it all and breathing out slow last night i was high i heard you say it was my body or my brain that said my soul was dead you can only argue with yourself for so long through my body or my brain i got myself insane hey yeah you got yourself all mixed up didn't you yeah didn't you yeah hey yeah you forgot that you gotta be taking in it all and breathing out slow ouch ahoh ahoh ahoh ahoh last night i was high i heard you say it was my body or my brain that said my soul was dead was it action or emotion said we're traveling in the ocean oh am i still alive there ain't no hell or heaven,0,/Kurt-vile-breathin-out-lyrics
17,Classic Rock In Spring/Freeway In Mind,hey how are ya you sure got a way of greeting a man might i add the perfect sun tan you're riding on your yellow schwinn while blasting classic rock in spring a couple of summer demons with battery recharging when you hear the bob seger song you know i'll be long gone cutting all my classes like a hit of acid hey how are ya like a dog on a telephone there is nonsense sustaining and staying on ya getting off you find yourself much better off feeling a bit less softer even but coughing up a lung and then some yeah you know i got your battery recharger right here and hey how are ya i got a freeway in mind let go of my head walk down my line better be sure you'll be dead i got a trumpet i know where to dump it i'm glad that you came if the sound is the same sometimes my reckless ways shock my self system for days now i'm channelling my faze in an anacin haze but it ain't gonna do me in i know when to dump it a hole in my way again i'll surely just jump it there was a kid in a tree among the birds and the bees between bee hive and bird nest and i think you know the rest he wanted to be free with them but they weren't believing pecking and stinging him till he wasn't breathing but it ain't gonna do him in he'll just go to heaven not done just a dreamer of ten or eleven,0,/Kurt-vile-classic-rock-in-spring-freeway-in-mind-lyrics
21,Damn You,instrumental,0,/Kurt-vile-damn-you-lyrics
25,Don’t Get Cute,don't get cute you know i can't compute competing with a man in an alligator suit gonna slap that man looking so damn daft but can i use you i wanna be a success give me my style back home on the range is a baby who has my eyes don't get cut you crazy little slut turning all the tricks in a twelveblock radius you may look cute but you puke all over it can i use you i wanna be a success give me my style back one mile back is my baby with the brown brown eyes don't get cute don't get cute don't get cute don't get caught you're a man not a fish and diving facefirst into a frozen lake baits with another don't 

In [226]:
unreleased_indices = list(np.where(mask)[0])

In [227]:
unreleased_indices

[7, 9, 13, 17, 21, 25, 27, 32, 43, 46, 52, 65, 74, 83, 100, 108, 118]

In [228]:
df.drop(unreleased_indices, inplace=True)

In [229]:
df.reset_index(drop=True, inplace=True)

In [230]:
df['release-year'].unique()

array([2013, 2015, 2009, 2011, 2018, 2016, 2022, 2020, 2010, 2019, 2021,
       2008])

In [232]:
# save processed dataframe as a CSV file
df.to_csv("/content/drive/MyDrive/Colab Notebooks/kv-lyrics-processed.csv")